In [1]:
import requests
from bs4 import BeautifulSoup

link = 'https://sweb.hku.hk/ccacad/ccc_appl/enrol_stat.html'
response = requests.get(link).text
doc = BeautifulSoup(response, 'html.parser')

rows = doc.find_all('tr')
courses = []
for row in rows:
    cols = row.find_all('td', {'bgcolor': '#FFFED1'})
    if cols:
        course = {
            'course_code': cols[0].font.text,
            'course_name': cols[1].font.text,
            'subclass' : cols[2].font.text,
            'quota': cols[3].font.text,
            'vacancies': cols[4].font.text,
            'waiting_applicants': cols[5].font.text,
        }
        
        courses.append(course)

In [2]:
import pandas as pd

cc_df = pd.DataFrame(courses)

cc_df['semester'] = 'First Semester'

cc_df.loc[98:, 'semester'] = 'Second Semester'

cc_df['AoI'] = cc_df['course_code'].str[2:4]

cc_df['quota'] = pd.to_numeric(cc_df['quota'])
cc_df['vacancies'] = pd.to_numeric(cc_df['vacancies'])
cc_df['waiting_applicants'] = pd.to_numeric(cc_df['waiting_applicants'])

cc_df['seats'] = cc_df['vacancies'] - cc_df['waiting_applicants']

cc_df.to_csv('2023Aug_Year2_CCEnrollmentStatistics.csv')

In [32]:
cc_df[['waiting_applicants','AoI']].groupby('AoI').sum()

,waiting_applicants
AoI,
CH,287
GL,450
HU,269
ST,266


In [40]:
# Course Searcher

cc_df.query("course_code == 'CCCH9007'")

,course_code,course_name,subclass,quota,vacancies,waiting_applicants,semester,AoI,seats
85,CCCH9007,China in the Global Economy (SCCE),A,64,64,29,First Semester,CH,35
187,CCCH9007,China in the Global Economy (SCCE),B,64,64,17,Second Semester,CH,47
